In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.axes
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Activation
import math
import tensorflow as tf
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import datetime 

OPEN-> 
CLOSE->
HIGH->
LOW->
PREV.CLOSE->
LTP->
CLOSE->
VWAP-> It's a trading benchmark that represents the average price a security has traded at throughout the day, based on both volume and price.
52W H->
52W L->	
VOLUME->
valteteteteteteteUE->
NO OF TRADES->


In [3]:
read_data_dir="D:\\programs\\Applied data Science\\proj\\cleaned"

In [4]:
ls=[]
files=[]
for root, dirs, files in os.walk(read_data_dir, topdown=False):
   for name in files:
      ls.append(os.path.join(root, name))
for i in range(len(files)):
   files[i]=files[i].replace(".csv","")

In [5]:
ls[29]

'D:\\programs\\Applied data Science\\proj\\cleaned\\KOTAKBANK.csv'

In [6]:
def ac_vs_pr(yac,ypr,title,option,file_index):
    plt.figure(figsize=(55,20))
    plt.title(title)
    plt.plot(np.arange(yac.size),ypr,label="ypredicted")
    plt.plot(np.arange(yac.size)+0.1,yac,label="yactual",alpha=0.6)
    plt.legend()
    plt.show()
    # plt.savefig("D:\\programs\\Applied data Science\\proj\\reg_res\\plots\\"+files[file_index]+" "+title+"_"+option+".png")

In [7]:
len(ls)

50

In [9]:
for v in range(49,50):
    #generating data
    df=pd.read_csv(ls[v])
    print(ls[v])
    dfr=pd.DataFrame(df[["close ","OPEN "]])
    dfr["date"]=pd.to_datetime(df["Date "])
    dfr=dfr.iloc[::-1]
    dfr.reset_index(inplace=True,drop=True)
    dfr["pct_chn"]=abs(dfr["OPEN "].shift(1)-dfr["OPEN "])/(dfr["close "]+dfr["OPEN "])
    dfr["logret"]=np.log(dfr["close "]) - np.log(dfr["close "].shift(1))
    dfr["pct_chn"][0]=0
    dfr["logret"][0]=0
    dfr
    #imputing data and cleaning up, saving
    for i in range(dfr["close "].size):
        if(dfr["pct_chn"][i]>0.3):
            dfr["pct_chn"][i]=dfr["pct_chn"].mean()
            dfr["logret"][i-1]=dfr["logret"].mean()
            print(dfr[i-3:i+3])
    dfr.to_csv("D:\\programs\\Applied data Science\\proj\\reg_res_logret\\data\\"+files[v]+".csv")
     # _=plt.figure(figsize=(25,10))
    # _=plt.plot(np.arange(dfr["close "].size),dfr["close "])
    # _=plt.xticks(ticks=np.arange(dfr["close "].size,step=50),labels=dfr["date"][::50],rotation=90)
    # _=plt.title(ls[v])

    time=90# time range to predict
    data=[]
    for i in range(time,dfr["OPEN "].size):
        data.append(np.array(dfr["logret"][i-time:i]))
    data=pd.DataFrame(data)
    dfr["OPEN "].size,data.shape
    split=math.ceil(data.shape[0]*0.8)

    #generating train and test
    x_train=[]
    y_train=[]
    for i in range(split-1):
        x_train.append(data.T[i])
        y_train.append(data.T[i+1])
    x_test=[]
    y_test=[]
    for i in range(split,data.shape[0]-1):
        x_test.append(data.T[i])
        y_test.append(data.T[i+1])
    x_train=np.array(x_train).reshape(split-1,time,1)
    x_test=np.array(x_test).reshape(data[0].size-split-1,time,1)
    y_train=np.array(y_train).reshape(split-1,time,1)
    y_test=np.array(y_test).reshape(data[0].size-split-1,time,1)
    x_train.shape,x_test.shape,y_train.shape,y_test.shape

    # building nueral network architecture
    rl=Sequential()
    rl.add(LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],1)))
    rl.add(Dropout(0.2,seed=170))
    rl.add(LSTM(units=50,return_sequences=True))
    rl.add(Dropout(0.2,seed=170))
    rl.add(LSTM(units=50,return_sequences=True))
    rl.add(Dropout(0.2,seed=170))
    rl.add(LSTM(units=50,return_sequences=True))
    rl.add(Dropout(0.2,seed=170))
    rl.add(LSTM(units=50,return_sequences=False))
    rl.add(Dropout(0.2,seed=170))
    rl.add(Dense(units=time))
    # rl.add(Activation('sigmoid'))

    # # fitting the nueral network
    rl.compile(optimizer="adam",loss="mean_squared_error",run_eagerly=False)
    rl.fit(x_train,y_train,epochs=200,batch_size=32)
    # rl=tf.keras.models.load_model("D:\\programs\\Applied data Science\\proj\\log_ret"+files[0]+".h5")
    op_te=rl.predict(x_test)
    op_tr=rl.predict(x_train)
    #modifying and selecting output
    prte=[]
    for i in range(0,op_te.shape[0],time):
        prte.append(op_te[i])
    prte=np.array(prte,dtype='object')
    prtr=[]
    for i in range(0,op_tr.shape[0],time):
        prtr.append(op_tr[i])
    prtr=np.array(prtr,dtype='object')
    valte=[]
    for i in range(0,prte.shape[0]):
        t=[]
        for j in range(prte[i].shape[0]):
            if(j==0):
                t.append(dfr["close "][split+i*time]*(math.e**prte[i][j]))
            else:
                t.append(math.e**prte[i][j]*t[j-1])
        valte.append(t)
    valte=pd.DataFrame(valte)
    valte
    valtr=[]
    for i in range(0,prtr.shape[0]):
        t=[]
        for j in range(prtr[i].shape[0]):
            if(j==0):
                t.append(dfr["close "][i*time]*(math.e**prtr[i][j]))
            else:
                t.append(math.e**prtr[i][j]*t[j-1])
        valtr.append(t)
    valtr=pd.DataFrame(valtr)
    valtr
    #generating plots, vaing plots and model
    plt.figure(figsize=(35,10))
    plt.plot(np.arange(dfr['close '][split:].size),dfr["close "][split:],label="actual data")
    for i in range(valte.shape[0]):
        plt.plot(np.arange(start=i*time,stop=(i+1)*time),valte.T[i],label="set "+str(i+1)+" 90 day predictions",alpha=0.8)
    plt.legend()
    plt.savefig("D:\\programs\\Applied data Science\\proj\\reg_res_logret\\plots\\Test"+files[v]+".png")
    plt.show()
    plt.figure(figsize=(35,10))
    plt.plot(np.arange(dfr['close '][:split].size),dfr["close "][:split],label="actual data")
    # cmap = matplotlib.colormaps['viridis']
    for i in range(valtr.shape[0]):
        plt.plot(np.arange(start=i*time,stop=(i+1)*time),valtr.T[i],label="set "+str(i+1)+" 90 day predictions",alpha=0.8)
    plt.legend()
    plt.savefig("D:\\programs\\Applied data Science\\proj\\reg_res_logret\\plots\\Train"+files[v]+".png")
    plt.show()
    rl.save("D:\\programs\\Applied data Science\\proj\\reg_res_logret\\model\\"+files[v]+".h5")

D:\programs\Applied data Science\proj\cleaned\WIPRO.csv
     close    OPEN        date   pct_chn    logret
976  545.90  549.90 2017-06-08  0.009400 -0.006391
977  536.95  544.05 2017-06-09  0.005412 -0.016531
978  526.35  535.60 2017-06-12  0.007957  0.000038
979  259.00  260.00 2017-06-13  0.006175 -0.709138
980  256.10  261.30 2017-06-14  0.002513 -0.011260
981  260.85  260.00 2017-06-15  0.002496  0.018378


C:\Users\Dell\AppData\Local\Temp\ipykernel_17220\4045682305.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr["pct_chn"][0]=0
C:\Users\Dell\AppData\Local\Temp\ipykernel_17220\4045682305.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr["logret"][0]=0
C:\Users\Dell\AppData\Local\Temp\ipykernel_17220\4045682305.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr["pct_chn"][i]=dfr["pct_chn"].mean()
C:\Users\Dell\AppData

Epoch 1/200
60/60 [==============================] - 16s 157ms/step - loss: 5.5860e-04
Epoch 2/200
60/60 [==============================] - 9s 154ms/step - loss: 5.5811e-04
Epoch 3/200
60/60 [==============================] - 10s 162ms/step - loss: 5.5776e-04
Epoch 4/200
60/60 [==============================] - 10s 160ms/step - loss: 5.5691e-04
Epoch 5/200
60/60 [==============================] - 10s 159ms/step - loss: 5.5596e-04
Epoch 6/200
15/60 [======>.......................] - ETA: 8s - loss: 4.5636e-04

KeyboardInterrupt: 

#### Model uses keras API from tensorflow
#### Model predicts based on close log ret 

# log(ti+1/ti) where i>=0 are known (assumed to be y(i+1))
# t0=x is known
# find t(i+1) for all relevant i's


log(t1/t0)=y1
t1/t0=e^y1
t1=t0*e^y1

log(t2/t1)=y2
log(t2/t1)+log(t1/t0)=y2+log(t1/t0)
log(t2/t0)=y2+y1
t2/t0=e^(y1+y2)
t2=t0*e^(y1+y2)


log(t3/t2)=y3
log(t3/t2)+log(t2/t1)+log(t1/t0)=y3+log(t1/t0)+log(t2/t1)
log(t3/t0)=y3+y2+y1
t3/t0=e^(y1+y2+y3)
t3=t0*e^(y1+y2+y3)